### Development Notebook for  `build_groundtruth_tf` function 
- Setup model, load weights and data generators 
- get items from data generator
- pass through network

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
import random
import math
import re
import  gc
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

import keras.backend as KB
sys.path.append('../')

import mrcnn.model_mod     as modellib
import mrcnn.visualize as visualize
import mrcnn.shapes    as shapes
from mrcnn.config      import Config
from mrcnn.model       import log
from mrcnn.dataset     import Dataset 

from mrcnn.utils       import stack_tensors, stack_tensors_3d
from mrcnn.datagen     import data_generator, load_image_gt
from mrcnn.callbacks   import get_layer_output_1,get_layer_output_2
from mrcnn.visualize   import plot_gaussian

# Root directory of the project
ROOT_DIR = os.getcwd()
MODEL_PATH = 'E:\Models'
# Directory to save logs and trained model
MODEL_DIR = os.path.join(MODEL_PATH, "mrcnn_development_logs")
# Path to COCO trained weights
COCO_MODEL_PATH   = os.path.join(MODEL_PATH, "mask_rcnn_coco.h5")
RESNET_MODEL_PATH = os.path.join(MODEL_PATH, "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")

print("Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))



# Build configuration object -----------------------------------------------
config = shapes.ShapesConfig()
config.BATCH_SIZE      = 3                  # Batch size is 2 (# GPUs * images/GPU).
config.IMAGES_PER_GPU  = 3                  # Must match BATCH_SIZE
config.STEPS_PER_EPOCH = 2
config.FCN_INPUT_SHAPE = config.IMAGE_SHAPE[0:2]
# config.display() 

# Build shape dataset        -----------------------------------------------
# Training dataset
# generate 500 shapes 
dataset_train = shapes.ShapesDataset()
dataset_train.load_shapes(150, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_train.prepare()
 
config.display()

try :
    del model, train_generator, val_generator, mm
    gc.collect()
except: 
    pass
# Load and display random samples
# image_ids = np.random.choice(dataset_train.image_ids, 3)
# for image_id in [3]:
#     image = dataset_train.load_image(image_id)
#     mask, class_ids = dataset_train.load_mask(image_id)
#     visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)
print(' COCO Model Path       : ', COCO_MODEL_PATH)
print(' Checkpoint folder Path: ', MODEL_DIR)
print(' Model Parent Path     : ', MODEL_PATH)
print(' Resent Model Path     : ', RESNET_MODEL_PATH)

import pprint
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4)

# batch_size = 3
# num_rois = 32
# num_classes = 4
# num_detections = 100
# num_rois_per_image = 32
# rois_per_image = 32

In [ ]:
try :
    del model
    print('delete model is successful')
    gc.collect()
except: 
    pass
KB.clear_session()
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [ ]:
print(' COCO Model Path       : ', COCO_MODEL_PATH)
print(' Checkpoint folder Path: ', MODEL_DIR)
print(' Model Parent Path     : ', MODEL_PATH)
print(' Resent Model Path     : ', RESNET_MODEL_PATH)
print(model.find_last())

# model.compile_only(learning_rate=config.LEARNING_RATE, layers='all')
# sys.setrecursionlimit(5000)
# tst = model.keras_model.to_json()
# save_model(MODEL_DIR, 'my_saved_model')
# print(model.find_last()

In [ ]:
#model.keras_model.summary(line_length = 120) 
# model.compile_only(learning_rate=config.LEARNING_RATE, layers='heads')
# KB.set_learning_phase(1)
'''
methods to load weights
1 - load a specific file
2 - find a last checkpoint in a specific folder 
3 - use init_with keyword 
'''
## 1- look for a specific weights file 
## Load trained weights (fill in path to trained weights here)
# model_path  = 'E:\\Models\\mrcnn_logs\\shapes20180428T1819\\mask_rcnn_shapes_5784.h5'
# print(' model_path : ', model_path )
# assert model_path != "", "Provide path to trained weights"
# print("Loading weights from ", model_path)
# model.load_weights(model_path, by_name=True)    
# print('Load weights complete')

# ## 2- look for last checkpoint file in a specific folder (not working correctly)
# model.config.LAST_EPOCH_RAN = 5784
# model.model_dir = 'E:\\Models\\mrcnn_logs\\shapes20180428T1819'
# last_model_found = model.find_last()
# print(' last model in MODEL_DIR: ', last_model_found)
# # loc= model.load_weights(model.find_last()[1], by_name=True)
# # print('Load weights complete :', loc)


## 3- Use init_with keyword
## Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
#     loc=model.load_weights(model.get_imagenet_weights(), by_name=True)
    loc=model.load_weights(RESNET_MODEL_PATH, by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    
    # See README for instructions to download the COCO weights
    loc=model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    loc= model.load_weights(model.find_last()[1], by_name=True)
print('Load weights complete')

In [ ]:
print('\n Inputs: ') 
for i, out in enumerate(model.keras_model.inputs):
    print(i , '    ', out)

print('\n Outputs: ') 
for i, out in enumerate(model.keras_model.outputs):
    print(i , '    ', out)


In [ ]:
train_generator = data_generator(dataset_train, model.config, shuffle=True,
                                 batch_size=model.config.BATCH_SIZE,
                                 augment = False)

In [ ]:
train_batch_x, train_batch_y = next(train_generator)

In [ ]:
# train_batch_x, train_batch_y = next(train_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [ ]:
# 0      Tensor("input_image:0", shape=(?, 128, 128, 3), dtype=float32)
# 1      Tensor("input_image_meta:0", shape=(?, ?), dtype=float32)
# 2      Tensor("input_rpn_match:0", shape=(?, ?, 1), dtype=int32)
# 3      Tensor("input_rpn_bbox:0", shape=(?, ?, 4), dtype=float32)
# 4      Tensor("input_gt_class_ids:0", shape=(?, ?), dtype=int32)
# 5      Tensor("input_gt_boxes:0", shape=(?, ?, 4), dtype=float32)
# 6      Tensor("input_gt_masks:0", shape=(?, 56, 56, ?), dtype=bool)

input_image_meta =  train_batch_x[1]
input_rpn_match  =  train_batch_x[2]
input_rpn_bbox   =  train_batch_x[3]
input_gt_class_ids = train_batch_x[4]
input_gt_bboxes = train_batch_x[5]
# gt_masks   =  train_batch_x[6]
print(' input_rpn_match    ', input_rpn_match.shape)
print(' input_rpn_bbox     ', input_rpn_bbox.shape)
print(' input_gt_class_ids ', input_gt_class_ids.shape)
print(' input_gt_bboxes    ', input_gt_bboxes.shape)


In [ ]:

model_output = get_layer_output_2(model.keras_model, train_batch_x, 1)


In [ ]:
print(len(model_output))
# rpn_class_logits   = model_output[0]
# rpn_class          = model_output[1]
# rpn_bbox           = model_output[2]
# rpn_proposal_rois  = model_output[3]
output_rois        = model_output[4]
target_class_ids   = model_output[5]
target_bbox_deltas = model_output[6]
roi_gt_boxes       = model_output[7]
mrcnn_class_logits = model_output[8]
mrcnn_class        = model_output[9]
mrcnn_bbox         = model_output[10]
# rpn_class_loss   = model_output[11]
# rpn_bbox_loss    = model_output[12]
# mrcnn_class_loss = model_output[13]
# mrcnn_bbox_loss  = model_output[14]
fcn_bbox_loss      = model_output[15]
pred_hm            = model_output[16]
gt_hm              = model_output[17]
pred_hm_norm       = model_output[18]
gt_hm_norm         = model_output[19]
pred_tensor        = model_output[20]
gt_tensor          = model_output[21]
gt_deltas          = model_output[22]
fcn_heatmap        = model_output[23]
fcn_class_logits   = model_output[24]
fcn_scores         = model_output[25]
fcn_bbox_deltas    = model_output[26]
# print(type(model_output[4]))
# print(type(output_rois))

In [ ]:
# print(target_bbox_deltas.shape)
# print(output_rois[0])
# print(roi_gt_boxes[0])
# print(target_class_ids[0])
# print(gt_deltas[0])
# print(target_bbox_deltas[0])
# print(pred_tensor[0,1])
# print(output_rois[0])
# print(mrcnn_class.shape)
# print(mrcnn_class[0])
# print(np.argmax(mrcnn_class[0], axis = -1))
print(target_class_ids[0])
print(target_bbox_deltas[0])
print(gt_deltas[0,2])

##  `development build_grround_truth_tf ()` 

In [ ]:
sess = KB.get_session()
gt_bboxes    = tf.identity(target_bbox_deltas)
gt_class_ids = tf.identity(target_class_ids)
num_bboxes   = 32

In [ ]:
# // pass model to TensorBuilder
batch_size      = config.BATCH_SIZE
num_classes     = config.NUM_CLASSES
h, w            = config.IMAGE_SHAPE[:2]
num_cols        = 6

# num of bounding boxes is determined by bbox_list.shape[1] instead of config.DETECTION_MAX_INSTANCES
# use of this routine for both input_gt_boxes, and target_gt_deltas
if num_bboxes == config.DETECTION_MAX_INSTANCES:
    tensor_name = "gt_tensor"
else:
    tensor_name = "gt_deltas"

print('\n')
print('  > BUILD_GROUND TRUTH_TF()' )
print('    gt_class_ids shape : ', KB.int_shape(gt_class_ids), '    gt_bboxes.shape  : ', gt_bboxes.get_shape() )

In [ ]:
#---------------------------------------------------------------------------
# use the argmaxof each row to determine the dominating (predicted) class
#---------------------------------------------------------------------------
# gt_classes     = gt_class_ids    # batch_size x max gt detections
gt_classes_exp = tf.to_float(tf.expand_dims(gt_class_ids ,axis=-1))
print('    gt_classes_exp shape ', gt_classes_exp.get_shape() )

ones = tf.ones_like(gt_class_ids)
zeros= tf.zeros_like(gt_class_ids)
mask = tf.greater(gt_class_ids , 0)
print(mask.shape)
print(mask.eval(session=sess))
gt_scores  =  tf.where(mask, ones, zeros)
# pred_scores      = tf.reduce_max(mrcnn_class ,axis=-1, keep_dims=True)   # (32,)
gt_scores_exp = tf.to_float(tf.expand_dims(gt_scores, axis=-1))
print('    gt_scores shape ', gt_scores.get_shape())
print(gt_scores.eval(session=sess))    


In [ ]:
#---------------------------------------------------------------------------
# create meshgrid to do something 
#---------------------------------------------------------------------------
batch_grid, bbox_grid = tf.meshgrid( tf.range(batch_size    , dtype=tf.int32), 
                                    tf.range(num_bboxes, dtype=tf.int32), indexing = 'ij' )

# sequence id is used to preserve the order of rois as passed to this routine
sequence = gt_scores * (bbox_grid[...,::-1] + 1) 
sequence = tf.to_float(tf.expand_dims(sequence, axis = -1))   

print('    batch_grid shape  ', batch_grid.get_shape())
print(batch_grid.eval(session=sess))
print('    bbox_grid  shape  ', bbox_grid.get_shape())
print(bbox_grid.eval(session=sess))
print('    sequence shape    ', sequence.get_shape())
print(sequence.eval(session=sess)) 

gt_array        = tf.concat([gt_bboxes, gt_classes_exp, gt_scores_exp, sequence ], axis=2)

print('    gt_array shape    ', gt_array.get_shape())
print(gt_array.eval(session=sess)) 
print(gt_array.shape[-1])    
# print(gt_array[:,:10].eval(session=sess))


In [ ]:
#------------------------------------------------------------------------------
# Create indicies to scatter rois out to multi-dim tensor by image id and class
# resulting tensor is batch size x num_classes x num_detections x 7 (num columns)
#------------------------------------------------------------------------------
scatter_ind = tf.stack([batch_grid , gt_class_ids, bbox_grid],axis = -1)
print('    scatter_ind shape ', scatter_ind.get_shape())
print(scatter_ind.eval(session=sess))


gt_scatter = tf.scatter_nd(scatter_ind, gt_array, [batch_size, num_classes, num_bboxes, gt_array.shape[-1] ])


# print('-- stack results ----')
print('    scatter_ind shape ', scatter_ind.get_shape())
# print(scatter_ind.eval())
print('    gt_scatter shape ', gt_scatter.get_shape())

print(gt_scatter[1,:,:,:].eval(session=sess))

In [ ]:
#-------------------------------------------------------------------------------
## sort in each class dimension based on y2 (column 1)
# scatter_nd places bboxs in a sparse fashion --- this sort is to place all bboxes
# at the top of the class bbox array
#-------------------------------------------------------------------------------
_ , sort_inds = tf.nn.top_k(gt_scatter[:,:,:,-1] , k=gt_scatter.shape[2])

# build indexes to gather rows from pred_scatter based on sort order 
class_grid, batch_grid, bbox_grid = tf.meshgrid(tf.range(num_classes),tf.range(batch_size), tf.range(num_bboxes))
bbox_grid_exp = tf.to_float(tf.expand_dims(bbox_grid, axis = -1))

print('    build gathering indexes to use in sorting -------')    
print('    sort inds shape : ', sort_inds.get_shape())
print('    class_grid  shape ', class_grid.get_shape())
# print(class_grid.eval())
print('    batch_grid  shape ', batch_grid.get_shape())
# print(batch_grid.eval())
print('    bbox_grid   shape ', bbox_grid.get_shape() , ' bbox_grid_exp shape ', bbox_grid_exp.get_shape())
# print(bbox_grid.eval())

gather_inds = tf.stack([batch_grid , class_grid, sort_inds],axis = -1)
gt_tensor   = tf.gather_nd(gt_scatter, gather_inds, name  = tensor_name)[...,:-1]
print('    gather_inds shape      : ', gather_inds.get_shape())
print('    gt_tensor (gathered)   : ', gt_tensor.get_shape())

In [ ]:
print(gt_tensor[2].eval(session=sess))
print(target_bbox_deltas[2])

In [ ]:
# append an index to the end of each row --- commented out 30-04-2018
# gt_tensor   = tf.concat([gt_tensor, bbox_grid_exp], axis = -1)

# count based on pred score > 0 (changed from index 0 to -1 on 30-04-2018)    
gt_cls_cnt  = tf.count_nonzero(gt_tensor[:,:,:,-1],axis = -1, name = 'gt_cls_count')

print('    final gt_tensor shape  : ', gt_tensor.get_shape())
print('    final gt_cls_cnt shape : ', gt_cls_cnt.get_shape())
print('    ', gt_cls_cnt.eval(session=sess))
print('    complete')

# return  [gt_tensor, gt_cls_cnt] 

In [ ]:
print(input_rpn_bbox.shape)
print(target_bbox_deltas[0])
print(gt_bboxes[0])
print(gt_class_ids[0])
print(gt_tensor[0,2].eval(session=sess))


In [ ]:
print(target_class_ids.shape)
sess = KB.get_session()
with sess.as_default():
    print(tf.where(target_class_ids > 0).eval())
# target_class_ids[0]
# target_bbox_deltas[0]